#### Esteganografia
O objetivo deste trabalho ´e implementar um algoritmo de esteganografia em imagens digitais.

In [1]:
import numpy as np
import cv2 as cv

Variables

In [2]:
file_image = "./images/baboon"
file_ext = ".png"
text_init = "Jesamin"
bit_plane = 0

In [3]:
def code(input, extension, text, plane):
    img = cv.imread(input + extension, cv.IMREAD_COLOR)

    text += chr(0)
    max = (img.shape[0] * img.shape[1] * 3) // 8
    if len(text) > max:
        raise ValueError("Insufficient bytes")

    ascii = np.frombuffer(text.encode(), 'S1').view(np.uint8)
    bits = np.unpackbits(ascii)

    out_img = img.ravel()
    out_txt = out_img[:len(bits)]

    # I can set 1 using x = x | pos or x = x & ~pos
    mask_a = 1 << plane
    mask_b = ~mask_a
    out_img[:len(bits)] = np.where(bits, mask_a | out_txt, mask_b & out_txt)

    output = out_img.reshape(img.shape)
    cv.imwrite(input + "_encode" + extension, output)
    


In [4]:
a = 63
k = 2
b = 0

c = a & ~1
d = c | b
e = (a & ~1) | b

f = (a & ~(1 << k)) | (b << k)

print(a, '{0:b}'.format(a))
print(c, '{0:b}'.format(c))
print(d, '{0:b}'.format(d))
print(e, '{0:b}'.format(e))
print(f, '{0:b}'.format(f), '\n')

ls = 3
print('{0:b}'.format(1 << ls))
print('{0:b}'.format(np.left_shift(1, ls)))

inv = 64
print('{0:b}'.format(inv))
print('{0:b}'.format(np.invert(inv)))
print('{0:b}'.format(~inv))


63 111111
62 111110
62 111110
62 111110
59 111011 

1000
1000
1000000
-1000001
-1000001


In [5]:
def decode(input, plane):
    # read image in RGB channel
    img = cv.imread(input, cv.IMREAD_COLOR)

    # get k lsb
    k = 1 << plane
    bits = (img & k).ravel() // k

    # flatten bits to bytes
    bits = np.reshape(bits, (-1, 8))
    values = [128, 64, 32, 16, 8, 4, 2, 1]
    bytes = np.sum(bits * values, axis=1)

    # make output
    output = []
    for b in bytes:
        if b == 0:
            break
        output.append(chr(b))
    return ''.join(output)

In [6]:
# code texto into image
code(file_image, file_ext, text_init*10, bit_plane)

In [7]:
decode(file_image + "_encode" + file_ext, bit_plane)

'JesaminJesaminJesaminJesaminJesaminJesaminJesaminJesaminJesaminJesamin'